If ever necesary, this part can be done following https://towardsdatascience.com/mapping-with-matplotlib-pandas-geopandas-and-basemap-in-python-d11b57ab5dac

In [1]:
## Import libraries
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from mlxtend.plotting import plot_decision_regions
import shapefile as shp
import seaborn as sns
from zipfile import ZipFile

In [3]:
## Data Cleaning
# Get energy dummy
sample_data = pd.read_excel('../EXAM/data/cleaned_data_excel.xlsx')
df = pd.DataFrame(sample_data)
df['energy'] = df['energy'].str.replace('Energimærke ', '')
df['energy'] = df['energy'].str.replace('A2015','A')
df['energy'] = df['energy'].str.replace('A2020','A')
dummy_energy = pd.get_dummies(df['energy'])
df = pd.concat([df,dummy_energy],axis=1)

# Change constructing year to construction age

df['age'] = 2023 - df['year']


In [7]:
## Dataset split
columns_to_drop = ['price','address','city','type','energy', 'year','area_name']
X,y = df.drop(columns_to_drop,axis=1),df.price
test_size = 0.30 # taking 70:30 training and test set
seed = 17082023 # Random number seeding for repeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
# Standardize X
norm_scaler = StandardScaler().fit(X_train)
X_train_std = norm_scaler.transform(X_train)
X_test_std = norm_scaler.transform(X_test)
X_train_std = pd.DataFrame(X_train_std,columns=X.columns)
X_test_std = pd.DataFrame(X_test_std,columns=X.columns)
X_std = pd.concat([X_train_std,X_test_std],axis=1)

      saledays  living_space  ground_space  rooms  owner_expenses  area_code  \
0          202           170          1019      2            2598       4591   
1          255           248           984      8            8643       2900   
2            0           163           858      5            3343       7100   
3            0           130           649      4            4974       2740   
4            0           173          1335      7            2673       8930   
...        ...           ...           ...    ...             ...        ...   
3001        48           109           506      3            1922       5250   
3002        48           117           800      3            1566       7620   
3003       240            86           103      4            2184       3320   
3004        48            79           829      3            1464       7160   
3005        48           133           129      5            4615       8240   

      Andelsbolig  Ejerlejlighed  Lande

In [5]:
## Initialize visualization set
sns.set(style='whitegrid', palette='pastel', color_codes=True)
sns.mpl.rc('figure', figsize=(10,6))

In [6]:
#opening the vector map
with ZipFile('Denmark_shapefile.zip') as shpfile:
    shpfile.extractall(path='../EXAM/Denmark_shapefile')
shp_path = '../EXAM/Denmark_shapefile/dk_1km.shp'
#reading the shape file by using reader function of the shape lib
sf = shp.Reader(shp_path)
len(sf.shapes())
# sf.records()
sf.records()[1][0]


'1kmE3886N3680'